In [1]:
import json

from web3 import Web3, HTTPProvider

In [2]:
# web3.py instance
w3 = Web3(HTTPProvider("http://127.0.0.1:8545"))

In [3]:
# account/contract data
account_address = "0x0026982384f0a6c6622a01a049f9fcdcab8f0c03"
account_checksum_address = w3.toChecksumAddress(account_address)
account_password = "password"
contract_address = "0xC9AC6dd6ea4D3583E0aed9D8d6D4d4380BF4bB88"

In [4]:
publication_data = {
    "title": "Breakthrough of the Year!",
    "description": "Phenomenal description",
    "author": "mr. Very Smart Scientist",
    "hash": "ed8a1bc7221493fe5e6d7bc19f035725e19a379q"
}

In [5]:
# load abi
abi_file = open('../contracts/publication/abi.json', 'r')
abi = json.load(abi_file)

In [6]:
# prepare contract
contract = w3.eth.contract(address=contract_address, abi=abi)

In [7]:
# check if the contract exists
w3.eth.getCode(contract.address)

HexBytes('0x608060405234801561001057600080fd5b5060043610610073577c0100000000000000000000000000000000000000000000000000000000600035046371838675811461007857806377f48119146102c55780638da5cb5b146104af578063981603da146104e0578063ef7922e3146104e8575b600080fd5b6102b36004803603608081101561008e57600080fd5b8101906020810181356401000000008111156100a957600080fd5b8201836020820111156100bb57600080fd5b803590602001918460018302840111640100000000831117156100dd57600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600092019190915250929594936020810193503591505064010000000081111561013057600080fd5b82018360208201111561014257600080fd5b8035906020019184600183028401116401000000008311171561016457600080fd5b91908080601f01602080910402602001604051908101604052809392919081815260200183838082843760009201919091525092959493602081019350359150506401000000008111156101b757600080fd5b8201836020820111156101c957600080fd5b803590602001918460018302840111640100000000831117156101eb57600080fd5b9190

In [8]:
# prepare account
w3.personal.unlockAccount(account_checksum_address, account_password)

True

In [9]:
# prepare contract function and estimate gas
add_publication_obj = contract.functions.addPublication(**publication_data)
gas_limit = add_publication_obj.estimateGas(
    {'from': w3.toChecksumAddress(account_checksum_address)}
)

In [10]:
# transact data to blockchain with calculated gas limit
add_publication_tx = add_publication_obj.transact({
    "gas": gas_limit,
    "from": account_checksum_address,
})
transaction_hash = add_publication_tx.hex()
print(transaction_hash)

0x9e6abdeb41c617fbcc58dd5594db41f0116f8b46ca04f6dc45f1be1ab24f3deb


In [11]:
# Wait for transaction to be mined...
print(w3.eth.waitForTransactionReceipt(transaction_hash))

AttributeDict({'blockHash': HexBytes('0x61482f6789271a7f6e81c91aeeb7406f14b195e474ac321bee0e1a4494d6ad88'), 'blockNumber': 5110582, 'contractAddress': None, 'cumulativeGasUsed': 1055451, 'from': None, 'gasUsed': None, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'root': None, 'status': 1, 'to': None, 'transactionHash': HexBytes('0x9e6abdeb41c617fbcc58dd5594db41f0116f8b46ca04f6dc45f1be1ab24f3deb'), 'transactionIndex': 13})


In [12]:
# try to transact with the same hash
w3.personal.unlockAccount(account_checksum_address, account_password)
add_same_publication = contract.functions.addPublication(**publication_data).transact()
print(w3.eth.waitForTransactionReceipt(add_same_publication.hex()))

AttributeDict({'blockHash': HexBytes('0x6879ba869541967c69fab2140c52ac3e3e49e268b7ba62320e432105a3d6c804'), 'blockNumber': 5110584, 'contractAddress': None, 'cumulativeGasUsed': 2625169, 'from': None, 'gasUsed': None, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'root': None, 'status': 0, 'to': None, 'transactionHash': HexBytes('0xa6a15ea3073e18cbc4f541eeb1e1585a94c4cc4b750dc8f48abe2d3590ab3516'), 'transactionIndex': 28})


In [13]:
# get publication by known hash
publication = contract.functions.getPublicationByHash(publication_data["hash"]).call()
print(publication)

['Breakthrough of the Year!', 'Phenomenal description', 'mr. Very Smart Scientist']


In [14]:
# get publication count to access the array of hashes
publication_cnt = contract.functions.getPublicationCount().call()
print(publication_cnt)

3


In [15]:
# get publication hash by index
publication_by_idx = contract.functions.getPublicationHash(0).call()
print(publication_by_idx)

ed8a1bc7221493fe5e6d7bc19f035725e19a379e
